## 1.0 - Base construction

### 1.1 - Import from libraries

In [ ]:
import pandas as pd

import requests
import json

import matplotlib.pyplot as plt
import seaborn as sns

### 1.2 - Customize figures

In [ ]:
sns.set_style('darkgrid')

color = 'deeppink'

## 2.0 - First steps

### 2.1 - Connection with the API

In [ ]:
url = 'https://pokeapi.co/api/v2/pokemon/'
response = requests.get(url)
content = response.json()
status_code = response.status_code

print('Connection status: {}'.format(status_code))
print('Total pokemons: {}'.format(content['count']))

### 2.2. - Pokemon names and address

In [ ]:
page = 'https://pokeapi.co/api/v2/pokemon/?offset=0&limit=20'
content['next'] = 0

poke_names = []

while content['next'] != None:
    response = requests.get(page)
    content = response.json()

    for i in content['results']:
        poke_names.append(i)

    page = content['next']

names = pd.DataFrame(poke_names)

### 2.3 - Getting the contents of pages

In [ ]:
poke_content = []

for i in names['url']:
    response = requests.get(i)
    content_json = response.json()

    poke_content.append(content_json)

content = pd.DataFrame(poke_content)

### 2.4 - Joining items 2.2 and 2.3

In [ ]:
pokemon = pd.concat([names, 
                     content.drop('name', axis= 1)], 
                    axis=1)

pokemon

## 3.0 - Manipulating dataframe

### 3.1 - Deleting uninteresting columns

In [ ]:
cols_delete = ['id', 'base_experience', 'url', 'forms', 'game_indices',
               'held_items', 'is_default', 'location_area_encounters',
               'order', 'species', 'sprites']
pokemon.drop(cols_delete, axis= 1, inplace= True)
pokemon

### 3.2 - Collecting column values

#### Column: abilities

In [ ]:
abilities = []

for i in pokemon['abilities']:
    ability = []
    
    for j in i:
        ability.append(j['ability']['name'])
        
    abilities.append(ability)

pokemon['abilities'] = abilities

#### Column: moves

In [ ]:
moves = []

for i in pokemon['moves']:
    move = []
    
    for j in i:
        move.append(j['move']['name'])
        
    moves.append(move)

pokemon['moves'] = moves

#### Column: stats

In [ ]:
stats = []

for i in pokemon['stats']:
    base_stat = []
    
    for j in i:
        base_stat.append(j['base_stat'])
        
    stats.append(base_stat)

stats_df = pd.DataFrame(stats,
                        columns= ['hp',
                                  'attack',
                                  'defense',
                                  'special-attack',
                                  'special-defense',
                                  'speed'])

pokemon = pd.concat([pokemon, stats_df], 
                    axis=1)

pokemon.drop('stats', axis= 1, inplace= True)

#### Column: types

In [ ]:
types = []

for i in pokemon['types']:
    type_ = []
    
    for j in i:
        type_.append(j['type']['name'])
        
    types.append(type_)

types = pd.DataFrame(types, 
                     columns= ['type_1', 'type_2'])

pokemon = pd.concat([pokemon, types], 
                    axis=1)

pokemon.drop('types', axis= 1, inplace= True)

#### Column: base_total

In [ ]:
pokemon['base_total'] = (pokemon['hp']
                         + pokemon['attack']
                         + pokemon['defense']
                         + pokemon['special-attack']
                         + pokemon['special-defense']
                         + pokemon['speed'])

#### Column: height

In [ ]:
pokemon['height'] = pokemon['height']/10

#### Column: weight

In [ ]:
pokemon['weight'] = pokemon['weight']/10

### 3.3 - Organizing the columns

In [ ]:
pokemon = pokemon[['name',
                   'height', 'weight',
                   'type_1', 'type_2',
                   'hp', 'attack',
                   'defense', 'special-attack',
                   'special-defense', 'speed',
                   'base_total', 'abilities',
                   'moves']]

### 3.4 - Finalized dataset

In [ ]:
pokemon

## 4.0 - Preliminary EDA

Before continuing, it would be very nice to take a look at the [dataset](https://www.kaggle.com/felipecabueno/complete-pokemon-database-by-pokeapico) page where you have all the detailed information for the columns.

### 4.1 - Weight vs Height

In [ ]:
fig, ax = plt.subplots(figsize = (10, 12))

plt.subplot(2,1,1)
sns.scatterplot(x= 'height',
                y= 'weight',
                data= pokemon,
                color= color)

plt.subplots_adjust(hspace= 0.3)
plt.show()

### 4.2 - View of types vs power

In [ ]:
fig, ax = plt.subplots(figsize = (10, 12))

plt.subplot(2,1,1)
sns.boxplot(x= 'type_1',
            y= 'base_total',
            data= pokemon,
            width= 0.3,
            color= color)
plt.xticks(rotation= 90)

plt.subplot(2,1,2)
sns.boxplot(x= 'type_2',
            y= 'base_total',
            data= pokemon,
            width= 0.3,
            color= color)
plt.xticks(rotation= 90)

plt.subplots_adjust(hspace= 0.3)
plt.show()

### 4.3 - Attributes distribution

In [ ]:
def dist_atr(dataframe, column):
    sns.kdeplot(dataframe[column],
                color= color)

fig, ax = plt.subplots(figsize = (10, 12))

plt.subplot(3,2,1)
dist_atr(pokemon, 'hp')

plt.subplot(3,2,2)
dist_atr(pokemon, 'attack')

plt.subplot(3,2,3)
dist_atr(pokemon, 'defense')

plt.subplot(3,2,4)
dist_atr(pokemon, 'special-attack')

plt.subplot(3,2,5)
dist_atr(pokemon, 'special-defense')

plt.subplot(3,2,6)
dist_atr(pokemon, 'speed')

plt.subplots_adjust(hspace= 0.3)
plt.show()

### 4.4 - Base total distribution

In [ ]:
fig, ax = plt.subplots(figsize = (10, 6))

dist_atr(pokemon, 'base_total')

plt.subplots_adjust(hspace= 0.3)
plt.show()

## 5.0 - Downloading the dataset

In [ ]:
pokemon.to_csv('pokemon.csv',
               index=False) 

The data input on this notebook was only useful to be linked with the [dataset](https://www.kaggle.com/felipecabueno/complete-pokemon-database-by-pokeapico). (The result)